# Séance 1 — Mathématiques pour l’informatique
**Thème :** Introduction & Logique (connecteurs, tables de vérité, équivalences) + Ensembles (bases)  
**Durée :** 90 minutes

> Ce notebook est clé-en-main pour animer la première séance : explications, démos Python, et exercices (avec vérifications automatiques).


## Objectifs d’apprentissage
- Formaliser des énoncés en **logique propositionnelle** et construire une **table de vérité**.  
- Utiliser des **équivalences** (De Morgan, implication ↔ disjonction) et raisonner sur la **tautologie**.  
- Introduire les idées de **preuve par contraposée** et **contradiction** (guidé).  
- Manipuler les **ensembles** (union, intersection, différence, cardinalité) et relier à des cas informatiques (doublons, `set`).

## Plan rapide
1. Rappels & mise en route (5–10 min)  
2. Logique : connecteurs, tables, équivalences (25–30 min)  
3. Preuves (contraposée/contradiction) — discussion guidée (10–15 min)  
4. Ensembles & applications en code (20–25 min)  
5. Activité binômes & ticket de sortie (10–15 min)


---
## 1) Logique propositionnelle — connecteurs & helpers

Nous utiliserons `True/False` en Python pour représenter la vérité d’une proposition.  
Définissons d’abord quelques fonctions utilitaires : `implies`, `iff` (équivalence), `xor`.


In [ ]:
def implies(p: bool, q: bool) -> bool:
    """Implication logique p -> q.
    Rappel : p -> q est équivalent à (non p) ou q.
    """
    return (not p) or q

def iff(p: bool, q: bool) -> bool:
    """Équivalence logique p <-> q (vrai si p et q ont la même valeur de vérité)."""
    return p == q

def xor(p: bool, q: bool) -> bool:
    """XOR logique (vrai si exactement un des deux est vrai)."""
    return (p and not q) or (not p and q)

# Petite vérification rapide
print("implies(False, False) =", implies(False, False))
print("iff(True, True)       =", iff(True, True))
print("xor(True, False)      =", xor(True, False))

---
## 2) Tables de vérité

Ci-dessous, des générateurs de tables pour 2 et 3 variables, et un vérificateur de tautologie.


In [ ]:
from itertools import product

def truth_table_2vars(expr, var_names=("p","q")):
    """Affiche la table de vérité d'une expression `expr(p, q)` pour p,q ∈ {False, True}.
    `expr` est une fonction à 2 arguments (bool, bool) -> bool.
    """
    p_name, q_name = var_names
    print(f"{p_name:>2} {q_name:>2} | expr")
    print("-"*14)
    for p, q in product([False, True], repeat=2):
        val = expr(p, q)
        print(f"{int(p):>2} {int(q):>2} | {int(val)}")

def truth_table_3vars(expr, var_names=("p","q","r")):
    """Affiche la table de vérité d'une expression `expr(p, q, r)` pour p,q,r ∈ {False, True}.
    `expr` est une fonction à 3 arguments (bool, bool, bool) -> bool.
    """
    p_name, q_name, r_name = var_names
    print(f"{p_name:>2} {q_name:>2} {r_name:>2} | expr")
    print("-"*20)
    for p, q, r in product([False, True], repeat=3):
        val = expr(p, q, r)
        print(f"{int(p):>2} {int(q):>2} {int(r):>2} | {int(val)}")

def is_tautology_2vars(expr) -> bool:
    """Renvoie True si l'expression expr(p,q) vaut 1 pour toutes les combinaisons de p,q."""
    return all(expr(p,q) for p,q in product([False, True], repeat=2))

def is_tautology_3vars(expr) -> bool:
    """Renvoie True si l'expression expr(p,q,r) vaut 1 pour toutes les combinaisons de p,q,r."""
    return all(expr(p,q,r) for p,q,r in product([False, True], repeat=3))

print("Helpers de tables chargés.")

### Exemple guidé
Montrer que \((p \rightarrow q) \leftrightarrow (\neg p \vee q)\) est une **tautologie**.


In [ ]:
# Définition de l'expression à 2 variables
expr = lambda p,q: iff(implies(p,q), (not p) or q)

# 1) Afficher la table de vérité
truth_table_2vars(expr)

# 2) Vérifier la tautologie
print("\nEst-ce une tautologie ? =>", is_tautology_2vars(expr))

### Pratique : équivalences utiles

- De Morgan : \(\neg(p \wedge q) \equiv \neg p \vee \neg q\) et \(\neg(p \vee q) \equiv \neg p \wedge \neg q\).  
- Biconditionnel : \(p \leftrightarrow q \equiv (p \wedge q) \vee (\neg p \wedge \neg q)\).

Exécutez la cellule suivante et vérifiez que ces équivalences sont des tautologies.


In [ ]:
# De Morgan (1) : ¬(p∧q) ↔ (¬p ∨ ¬q)
expr_demorgan1 = lambda p,q: iff(not (p and q), (not p) or (not q))

# De Morgan (2) : ¬(p∨q) ↔ (¬p ∧ ¬q)
expr_demorgan2 = lambda p,q: iff(not (p or q), (not p) and (not q))

# Biconditionnel : (p↔q) ↔ (p∧q)∨(¬p∧¬q)
expr_bicond = lambda p,q: iff(iff(p,q), (p and q) or ((not p) and (not q)))

print("De Morgan (1) tautologie ? =>", is_tautology_2vars(expr_demorgan1))
print("De Morgan (2) tautologie ? =>", is_tautology_2vars(expr_demorgan2))
print("Biconditionnel tautologie ? =>", is_tautology_2vars(expr_bicond))

---
## 3) Preuves (contraposée & contradiction) — discussion guidée

### Contraposée
Pour prouver \(p \rightarrow q\), on peut prouver sa **contraposée** : \(\neg q \rightarrow \neg p\).  
Exemple classique : *“Si \(n^2\) est pair, alors \(n\) est pair.”*  
Contraposée : *“Si \(n\) est impair, alors \(n^2\) est impair.”*

> **Exercice guidé :** Montrer que si \(n\) est impair (\(n=2k+1\)), alors \(n^2 = 4k(k+1)+1\) est impair.

### Contradiction
Supposer l’énoncé **faux**, déduire une impossibilité (contradiction), conclure que l’énoncé est **vrai**.

> **Exercice de réflexion :** Prouver par contradiction qu’il n’existe pas de plus petit rationnel strictement positif.


**Illustration numérique (impair ⇒ carré impair)**  
(Ne remplace pas une preuve !)

In [ ]:
def is_odd(n: int) -> bool:
    return n % 2 == 1

# Vérification pour quelques n impairs
tests = [1, 3, 5, 11, 21]
[(n, is_odd(n), is_odd(n*n)) for n in tests]

---
## 4) Ensembles — bases & applications

Rappels rapides :  
- Notations : \(A \cup B\) (union), \(A \cap B\) (intersection), \(A \setminus B\) (différence), \(A \triangle B\) (différence symétrique).  
- Cardinalités : \(|A \cup B| = |A| + |B| - |A \cap B|\).

Nous utiliserons les **ensembles Python** (`set`) pour illustrer.


In [ ]:
# Démo de base avec des ensembles Python
A = {1, 2, 3, 5}
B = {3, 4, 5, 6}

print("A =", A)
print("B =", B)
print("Union (A|B)         =", A | B)
print("Intersection (A&B)   =", A & B)
print("Différence (A-B)     =", A - B)
print("Diff. symétrique ^   =", A ^ B)

# Cardinalité de l'union (formule d'inclusion-exclusion)
len_union = len(A) + len(B) - len(A & B)
print("len(A∪B) par formule =", len_union, " / check =", len(A | B))

### Application pratique : détecter des doublons avec `set`

Si une collection contient un doublon, alors la taille de l’ensemble des éléments est **strictement inférieure** à la longueur de la collection.


In [ ]:
def has_duplicate(seq):
    """Renvoie True si la séquence contient au moins un doublon."""
    return len(set(seq)) < len(seq)

# Démonstration
emails = ["a@ex.com", "b@ex.com", "c@ex.com", "b@ex.com"]
nums   = [1,2,3,4,5]
print("emails:", emails, " -> doublon?", has_duplicate(emails))
print("nums  :", nums,   " -> doublon?", has_duplicate(nums))

### Exercice : Inclusion–Exclusion

Calculez \(|A \cup B|\) avec \(|A|=12, |B|=9, |A \cap B|=5\).  
*(Attendu : 16)* — vérifiez avec la fonction ci-dessous.


In [ ]:
def union_cardinality(lenA: int, lenB: int, len_inter: int) -> int:
    return lenA + lenB - len_inter

print("Test :", union_cardinality(12, 9, 5))

### Principe des tiroirs (pigeonhole)

> S’il y a plus d’objets que de tiroirs, au moins un tiroir contient ≥ 2 objets.

Exemple : parmi 13 personnes, deux partagent un mois de naissance (12 tiroirs → 13 objets).


---
## 5) Activité binômes (choisir A ou B)

**A. Logique**  
1. Construire la table de \((p\rightarrow q)\leftrightarrow(\neg p \vee q)\).  
2. Conclure : tautologie ? Pourquoi est-ce utile en code ?

**B. Ensembles**  
1. On vous donne une liste d’identifiants (avec doublons possibles). Écrire une fonction qui renvoie le nombre d’IDs **uniques**.  
2. Que vaut ce nombre par rapport à la longueur de la liste ? Dans quel cas d’égalité ?


### Aides (facultatives) pour vérifier vos réponses


In [ ]:
# Aide A.1 : (ré)afficher la table de la tautologie clé
truth_table_2vars(lambda p,q: iff(implies(p,q), (not p) or q))

# Aide B.1 : compteur d'uniques
def count_unique(seq):
    return len(set(seq))

print("count_unique(['id1','id1','id2']) =", count_unique(['id1','id1','id2']))

---
## 6) Mini-Quiz (auto-vérif)

**Q1.** V/F — \(p \rightarrow q \equiv \neg p \vee q\).  
**Q2.** Donner la **contraposée** de \(p \rightarrow q\).  
**Q3.** Simplifier \(\neg(\neg p \vee q)\) avec De Morgan.  
**Q4.** Écrire une fonction `has_duplicate` (déjà proposée) et tester sur 2 listes (une avec, une sans doublon).

> Utilisez la cellule suivante pour *cocher* vos réponses (via des `assert`).


In [ ]:
# Q1 (V/F) : on vérifie que c'est une tautologie
assert is_tautology_2vars(lambda p,q: iff(implies(p,q), (not p) or q))

# Q2 : contraposée (à écrire en commentaire pour s'auto-corriger)
contraposee = "¬q -> ¬p"  # remplacez si vous voulez tester votre mémoire
assert contraposee.replace(" ", "") in {"¬q->¬p", "~q->~p", "nonq->nonp"}  # tolérant

# Q3 : De Morgan : ¬(¬p ∨ q) ≡ p ∧ ¬q
expr_q3 = lambda p,q: iff(not((not p) or q), p and (not q))
assert is_tautology_2vars(expr_q3)

# Q4 : déjà fait plus haut; on re-teste rapidement
assert has_duplicate([1,2,3,2]) is True
assert has_duplicate([1,2,3,4]) is False

print("✅ Mini-Quiz : vérifications passées.")

---
## 7) Wrap-up & Ticket de sortie
- Écrivez en **une phrase** une équivalence logique que vous retiendrez aujourd’hui et pourquoi elle est utile en code.  
- (Facultatif) Notez une question pour la prochaine séance.

**Prochaine séance :** Ensembles, relations & fonctions (avec mini-projet Jupyter).


---
## Annexe — Outils supplémentaires (optionnel)

### Générateur générique pour 3 variables


In [ ]:
# Exemple : table pour (p∧q) → r
expr3 = lambda p,q,r: implies(p and q, r)
truth_table_3vars(expr3, var_names=("p","q","r"))